In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [3]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [4]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

c:\users\ashutosh\appdata\local\programs\python\python39\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(random_state=42)

In [5]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.8348666666666666

In [6]:
y_pred_test = lin_clf.predict(X_test)
accuracy_score(y_test, y_pred_test)

0.8236

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [8]:
svm_clf = SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC()

In [9]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9455333333333333

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ...C=4.324834213672926, gamma=0.0069477632349128635; total time=   0.3s
[CV] END ...C=4.324834213672926, gamma=0.0069477632349128635; total time=   0.2s
[CV] END ...C=4.324834213672926, gamma=0.0069477632349128635; total time=   0.2s
[CV] END ....C=5.373608963748895, gamma=0.005591541708670301; total time=   0.2s
[CV] END ....C=5.373608963748895, gamma=0.005591541708670301; total time=   0.2s
[CV] END ....C=5.373608963748895, gamma=0.005591541708670301; total time=   0.2s
[CV] END ....C=6.692407666414946, gamma=0.028357128925955207; total time=   0.2s
[CV] END ....C=6.692407666414946, gamma=0.028357128925955207; total time=   0.2s
[CV] END ....C=6.692407666414946, gamma=0.028357128925955207; total time=   0.2s
[CV] END ....C=5.977875028221859, gamma=0.001353853410537178; total time=   0.2s
[CV] END ....C=5.977875028221859, gamma=0.001353853410537178; total time=   0.2s
[CV] END ....C=5.977875028221859, gamma=0.001353

RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002058EF99700>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002058EF8DEB0>},
                   verbose=2)

In [11]:
rnd_search_cv.best_estimator_


SVC(C=5.977875028221859, gamma=0.001353853410537178)

In [12]:
rnd_search_cv.best_score_


0.8619907332482183

In [13]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)


SVC(C=5.977875028221859, gamma=0.001353853410537178)

In [14]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9729